In [100]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go 

In [ ]:
"""
NOTE: combining 'ORI' and 'Incident Number' is not enough for a unique identifier per event. 
Waiting to hear clarification form Ben, but could create a key using date also. That should
be enough to join matching entries from one table to another correctly.
NOTE: Reply: heard back from Ben, got a new csv from a specific PD that only has 198 records, of which only 170 have unique INs 
"""

In [101]:
# Deliverables 1 & 2 Need to join:
# Data/2020 Domestic Violence - Administrative Data.csv
# Data/2020 Domestic Violence - Other Data.csv
# Data/2020 Q2 Domestic Violence - Location Type.csv
admin_df = pd.read_csv('../Data/2020 Domestic Violence - Administrative Data.csv', encoding='ISO-8859-1',skiprows=7)
other_df = pd.read_csv('../Data/2020 Domestic Violence - Other Data.csv', encoding='ISO-8859-1',skiprows=7)
location_df = pd.read_csv('../Data/2020 Q1 Domestic Violence - Location Type.csv', encoding='ISO-8859-1',skiprows=6)
#nadmin_df = pd.read_csv('../Data/new2020 Domestic Violence - Administrative for Athens PD 08192022.csv', encoding='ISO-8859-1',skiprows=6)


In [102]:
# print(len(admin_df), len(admin_df.drop_duplicates()))
admin_df['Incident_Key'] = admin_df['ORI'] + admin_df['Incident Number'] + admin_df['Incident Date']
# print(len(admin_df['Incident_Key'].unique()))
# print(len(admin_df.drop_duplicates('Incident_Key')))

In [103]:
base_df = admin_df.drop_duplicates('Incident_Key')
# MM/DD/YYYY -> need Day and Month specifically
base_df[['Month', 'Day','Year']] = base_df['Incident Date'].str.split('/', expand=True)
base_df[['Hour','End']] = base_df['Hour of Day'].str.split('-', expand=True)
defects = {
    '11:0 0am':'11:00am',
    '12:00n':'12:00pm',
    }
base_df.replace(to_replace=defects, inplace=True)
base_df.reset_index(drop=True, inplace=True)
# dropping records without time stamps as this would not contribute to a timeseries analysis
base_df = base_df[base_df.Hour != 'Unknown']



/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_15935/2879768618.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_15935/2879768618.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_15935/2879768618.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [104]:
def change_time_format(x):
    in_time = datetime.strptime(x, "%I:%M%p")
    return datetime.strftime(in_time, "%H:%M")

def change_time_format2(x):
    n = int(x[:2])
    return n

def day_of_week(x):
    dow = datetime.strptime(x, "%m/%d/%y")
    return dow.strftime("%A")

def week_of_year(x):
    woy = datetime.strptime(x, "%m/%d/%y")
    return woy.isocalendar()[1]

def month_to_season(x):
    if x in [1,2,12]:
        return 'Winter'
    elif x in [3,4,5]:
        return 'Spring'
    elif x in [6,7,8]:
        return 'Summer' 
    else:
        return 'Fall'
    return 

In [ ]:
# test_date = base_df['Incident Date'][6000]
# d = datetime.strptime(test_date, "%m/%d/%y")
# print(test_date, d.strftime("%A"))
# yr, week_num, day_of_week = d.isocalendar()
# week_of_year(test_date)

In [105]:
base_df.Hour = base_df.Hour.apply(change_time_format)
base_df.Day = base_df.Day.apply(int)
base_df.Month = base_df.Month.apply(int)
base_df['Weekday'] = base_df['Incident Date'].apply(day_of_week)
base_df['Week_Num'] = base_df['Incident Date'].apply(week_of_year)

/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_15935/494222043.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_15935/494222043.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_15935/494222043.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [106]:
ts_df = base_df[['ORI',
    'Incident Number',
    'Incident Date',
    'Day',
    'Month',
    'Hour',
    'Weekday',
    'Week_Num',
    'Incident Clearance']
]

ts_df['Season'] = base_df.Month.apply(month_to_season)
# ts_df.head()


/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_15935/1746826813.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [109]:
X = ts_df.groupby(['Incident Date']).count()
X.to_csv('../Data/Mart2/fix.csv')

In [ ]:
# puts the dates in order

days = list(set(ts_df['Incident Date']))
days.sort(key=lambda date: datetime.strptime(date, "%m/%d/%y"))

# creating frequency of events per day
per_day = {}
X = ts_df.groupby(['Incident Date'])['Incident Date'].count()
for i in days:
    per_day[i] = X[i]

# creating frequency of events per month
per_month = {}
Y = ts_df.groupby(['Month'])['Month'].count()
for m in Y.index:
    per_month[m] = Y[m]

# creating frequency of events per day of the month over the year
per_day_of_month = {}
Z = ts_df.groupby(['Day'])['Day'].count()
for d in Z.index:
    per_day_of_month[d] = Z[d]

# creating frequecy of overall events per hour 
per_hour_overall = {}
H = ts_df.groupby(['Hour'])['Hour'].count()
for h in H.index:
    per_hour_overall[h] = H[h]

# creating frequency by hours per month, [0] = per year, in desc order
# Ex. get top 3 hours for february hours_by_month[2][:3]
# Ex. want standard deviation for March hours_by_month[3].describe()['std']
hours_by_month = {}
days_per_month = {}
for i in range(13):
    if i == 0:
        # over all year
        hours_by_month[i] = ts_df.Hour.value_counts()
        days_per_month[i] = ts_df.Day.value_counts()
    else:
        # by month
        filter = ts_df[ts_df.Month == i]
        monthly = filter['Hour'].value_counts()
        daily = filter['Day'].value_counts()
        hours_by_month[i] = monthly
        days_per_month[i] = daily

hours_by_season, days_per_season = {}, {}
for i in ['Winter', 'Spring', 'Summer', 'Fall']:
    filter = ts_df[ts_df.Season == i]
    hours = filter['Hour'].value_counts()
    daily = filter['Day'].value_counts()
    hours_by_season[i] = hours
    days_per_season[i] = daily



In [ ]:
# computing monthly...
# hourly average (remember that there are 31 instances of each hour over march, so > day average makes sense)
print(hours_by_month[3][:1])
# daily average
print(days_per_month[1].mean())

In [ ]:
# checking occurrences on January 1 against .loc
for j in range(1,13):

    for i in range(1,len(days_per_month[j])+1):
        n = days_per_month[j][i]
        t = ts_df.loc[ts_df.Month == j]['Day'].value_counts()[i]
        if n != t:
            print(f'Error at day {i}')

    print(f'Month {j} check complete.')

In [ ]:
# getting means and std dev of days and hours per month
to_plot = {
    'Month':[],
    'Mean':[],
    'StDev':[],
    'Median':[],
    'Num Days':[]
}
for i in range(1,13):
    dpm = days_per_month[i].describe()
    # get the month's daily average & stdev 
    avg = round(dpm['mean'],2)
    std = round(dpm['std'],2)
    to_plot['Month'].append(i)
    to_plot['Mean'].append(avg)
    to_plot['StDev'].append(std)
    to_plot['Num Days'].append(dpm['count'])
    to_plot['Median'].append(dpm['50%'])

# pd.DataFrame(to_plot).plot.line(x='Month',y=['Mean','StDev'])


In [ ]:
# boxplots per month
# trend line of mean and std per month



fig = go.Figure()
for i in range(1,13):
    date_object = datetime.strptime(str(i),'%m')
    mname = date_object.strftime('%b') 


    fig.add_trace(go.Box(
        y=days_per_month[i].values,
        x=[i]*len(days_per_month[i]),
        name=mname,
        marker_color='rgb(9,56,125)'
    ))

fig.update_layout(
    yaxis_title='Daily Instances of DA',
    title_text='Instances of DA per Day by Month'
)

fig.show()
fig.write_image('../Visuals/DA_Monthly_by_Day.png', engine="kaleido")

In [ ]:
def get_monthly_name(x):
    if x != 0:
        date_object = datetime.strptime(str(i),'%m')
        return date_object.strftime('%b')
    else:
        return 'Yearly Total'


# creating tabular format
table1 = pd.DataFrame(days_per_month)
for i in range(13):
    try:
        table1.rename({i:get_monthly_name(i)},axis=1, inplace=True)
    except ValueError:
        print(i)

table1.replace({np.NAN:''}, inplace=True)
table1_summary = table1.describe()
# table1.tail()
table1.to_csv('../Tables/T1:DA_Monthly_by_Day.csv')


In [ ]:
# creating special case since shorter months were ommitted from .describe() b/c inconsistent len

missed_months = ['Feb', 'Apr','Jun','Sep','Nov']
missed = {
    'Month':[],
    'count':[],
    'mean':[],
    'std':[],
    'min':[],
    '25%':[],
    '50%':[],
    '75%':[],
    'max':[]
}
for mo in missed_months:
    if mo == 'Feb':
        n = 29
    else:
        n = 30
    missed['Month'].append(mo)
    missed['count'].append(n)
    missed['mean'].append(table1[mo][:n].mean())
    missed['std'].append(table1[mo][:n].std())
    missed['min'].append(table1[mo][:n].min())
    missed['25%'].append(table1[mo][:n].quantile(q=.25))
    missed['50%'].append(table1[mo][:n].quantile(q=.5))
    missed['75%'].append(table1[mo][:n].quantile(q=.75))
    missed['max'].append(table1[mo][:n].max())

X = pd.DataFrame(missed).T
cols = list(X.loc['Month'].values)
for i in range(5):
    X.rename({i:cols[i]}, axis=1,inplace=True)
X.drop('Month',axis=0, inplace=True)
table1_summary = pd.concat([table1_summary,X], axis=1)
table1_summary = table1_summary[['Yearly Total', 'Jan', 'Feb', 'Mar', 'Apr', 'May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']]
table1_summary = table1_summary.astype(float)
table1_summary.to_csv('../Tables/T1_Summary.csv')

In [ ]:
# currently, the seasonal by day plot aggregates the days of the month. This is misleading. Instead,
# can we accumulate data points per season for each day in that season. 
mo_in_season = {
    'Winter': [1,2,12],
    'Spring': [3,4,5],
    'Summer': [6,7,8],
    'Fall': [9,10,11]
}
season_total = {
        'X':[],
        'Y':[],
        'Season':[]
    }
for season in ['Winter', 'Spring', 'Summer', 'Fall']:
    # i.e. pull months for season...
    # get days
    for month in mo_in_season[season]:
        # pull days for each of those months
        for day in days_per_month[month]:
            season_total['X'].append(month)
            season_total['Y'].append(day)
            season_total['Season'].append(season)



In [ ]:
# Winter [:91], Spring [92:183], Summer [184:275], Fall [276:]
# print(season_total['Season'].count('Summer'))
# print(183+91)
# season_total['X'][276:]
Seasonal_df = pd.DataFrame(season_total)

In [ ]:
fig_season = go.Figure()
for i in ['Winter', 'Spring', 'Summer', 'Fall']:

    fig_season.add_trace(go.Box(
        y=Seasonal_df[Seasonal_df['Season'] == i]['Y'],
        x=Seasonal_df[Seasonal_df['Season'] == i]['Season'],
        name=i,
        marker_color='rgb(9,56,125)'
    ))

fig_season.update_layout(
    yaxis_title='Daily Instances of DA',
    title_text='Instances of DA per Day by Season'
)

fig_season.show()
fig_season.write_image('../Visuals/DA_Season_by_Day.png', engine="kaleido")

In [ ]:
# NOTE: running this will remove Winter-Mod from table2 summary

# # creating tabular format
# # table2 is Seasonal_df
# table2_summary = Seasonal_df[Seasonal_df.Season == 'Winter'].describe().rename({'Y':'Winter'}, axis=1).drop('X', axis=1)
# for s in ['Spring','Summer','Fall']:
#     X = Seasonal_df[Seasonal_df.Season == s].describe().rename({'Y':s},axis=1).drop('X',axis=1)
#     table2_summary = pd.concat([table2_summary, X], axis=1)
# Seasonal_df.to_csv('../Tables/T2:DA_Seasons_by_Day.csv')
# table2_summary.to_csv('../Tables/T2_Summary.csv')

In [ ]:
# creating frequency distribution per day of the month
day_df = pd.DataFrame(ts_df.groupby(['Day'])['Day'].count())
print(f"Average Incidents per day of the month: {ts_df.groupby(['Day'])['Day'].count().mean()}")
fig_day = px.bar(day_df, 
    x=day_df.index, 
    y='Day', 
    labels={'index':'Day of the Month', 'Day':'Count'},
    title='Incident Frequency Per Day of Month over the Year')
fig_day.show()

fig_day.write_image('../Visuals/Daily_total_over_year.png', engine="kaleido")

In [ ]:
# creating tabular format
day_df.to_csv('../Tables/T3:Total_Frequency_by_Day_of_Month.csv')
day_df_summary = day_df.describe()
day_df_summary.to_csv('../Tables/T3_Summary.csv')

In [ ]:
# distribution of incidents per day of the week
weekday_df = pd.DataFrame(ts_df.groupby(['Weekday'])['Weekday'].count())
print(f"Average Incidents per day of week: {ts_df.groupby(['Weekday'])['Weekday'].count().mean()}")
fig_weekday = px.bar(weekday_df, 
    x=weekday_df.index, 
    y='Weekday', 
    labels={'index':'Day of the Week', 'Weekday':'Count'},
    title='Incident Frequency Per Day of Week over the Year')
fig_weekday.show()

fig_weekday.write_image('../Visuals/Weekday_total_over_year.png', engine="kaleido")

In [ ]:
weekday_overtime = {
    'Month':[],
    'Weekday average':[],
    'Weekday Monthly Total':[],
    'Weekday':[],
    # 'Week_Num':[]
}

# iterate over the months
for i in range(1,13):
    # iterate over the days of the week
    month_slice = ts_df.loc[ts_df.Month == i]
    for day in ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']:
        weekday_overtime['Month'].append(i)
        weekday_slice = month_slice.loc[month_slice.Weekday == day]
        weekday_overtime['Weekday'].append(day)
        # total frequency of incidents over month on day of week
        total = len(weekday_slice)
        weekday_overtime['Weekday Monthly Total'].append(total)
        # number of weekdays that occur in month
        num = len(weekday_slice['Incident Date'].unique())
        #average weekly occurrence
        weekday_overtime['Weekday average'].append(round(float(total)/num,2))
        

In [ ]:
weekday_per_month = pd.DataFrame(weekday_overtime)
# wpm = px.line(weekday_per_month,
#     x='Month',
#     y='Weekday average',
#     color='Weekday',  
#     symbol='Weekday',
#     # trendline='ols',
#     # trendline_scope='overall' 

wpm = go.Figure()

for day in weekday_per_month.Weekday.unique():
    dfwpm = weekday_per_month[weekday_per_month.Weekday == day]
    wpm.add_trace(go.Scatter(
        x= dfwpm['Month'],
        y= dfwpm['Weekday average'],
        mode='lines',
        name= day + '_avg'
    ))
    # wpm.add_trace(go.Scatter(
    #     x=dfwpm['Month'],
    #     y=dfwpm['Weekday Monthly Total'],
    #     mode='lines',
    #     name=day + '_total'
    # ))
wpm.update_layout(
    xaxis_title='Month',
    yaxis_title='Instances',
    title_text='Montly average per weekday'
)
print(f"{ts_df['Weekday'][0]} is the day of the week of Jan 1")
wpm.show()
wpm.write_image('../Visuals/Weekday_average_per_month.png', engine="kaleido")

In [ ]:
# table 8: Weekday per month
weekday_per_month


In [ ]:
# getting weekly report
def get_weekly(week_num, df=ts_df):
    from statistics import stdev

    days_of_week = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
    # get week's slice from df
    slice = df[df.Week_Num == week_num]
    # get TOTAL occurrences for the week
    week_total = len(slice)
    day_totals = []
    for day in days_of_week:
        day_freq = len(slice.loc[slice.Weekday == day])
        day_totals.append(day_freq)

    # NOTE: this will hide days that share max values, if Sunday and Friday have the max, only sunday will be reported
    # Weekly MAX
    day_max = max(day_totals)
    # DAY of MAX
    day_of_max = days_of_week[day_totals.index(day_max)]
    # Weekly AVERAGE
    week_avg = round(sum(day_totals)/len(day_totals),4)
    # STDEV of week
    sd = round(stdev(day_totals,week_avg),4)
    # SEASON
    season = slice.Season.unique()[0]

    return week_total, day_of_max, day_max, week_avg, sd, season
    

In [ ]:
# create a line plot that tracks weekday and weekly totals over the 53 weeks of the leap year
# need weekly summary: weekly frequency of DV instances, average, st dev
# len(ts_df.Week_Num.unique()) # 53
# instantiate graph_object Figure
table9 = {
    'Week_Num':[],
    'Weekly Total':[],
    'Weekly Average': [],
    'Weekly Stdev':[],
    'Weekly Max':[],
    'Weekday of Max':[],
    'Season': []
}

# weekly_fig = go.Figure()

for week_num in range(1,54):
    # week of year
    table9['Week_Num'].append(week_num)
    # generate report
    total, day_of_max, day_max, week_avg, week_std, season = get_weekly(week_num=week_num)
    table9['Weekly Total'].append(total)
    table9['Weekday of Max'].append(day_of_max)
    table9['Weekly Max'].append(day_max)
    table9['Weekly Average'].append(week_avg)
    table9['Weekly Stdev'].append(week_std)
    # grab season for coloring of figure
    table9['Season'].append(season)

In [ ]:
# occurrences per week
weekly_report = pd.DataFrame(table9)
weekly_fig = px.line(weekly_report,
    x='Week_Num',
    y='Weekly Total',
    # color='Season',  
    # symbol='Weekday',
    # trendline='ols',
    # trendline_scope='overall'
) 
weekly_fig.update_layout(
    title_text = 'Occurrences of DV per Week'
)
weekly_fig.show()
# weekly_fig.write_image('../Visuals/Weekly_max.png', engine="kaleido")

In [ ]:
weekly_report.head()

In [ ]:
def make_theta_from_weeknum(Week_Num):
    # ranges from wk 1 to wk ...
    last_week_num = weekly_report.Week_Num.max()
    scalar = 360/last_week_num
    return Week_Num * scalar

In [ ]:
# Creating a radial histogram for weekly data in place of figure 9
t = list(weekly_report.Week_Num.astype(str))

radialfig = px.bar_polar(weekly_report, r='Weekly Total', theta=t,
    width=800, height=800,
    color='Season', template="plotly_dark",
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    title='Frequency of DV per Week of the Year')

radialfig.show()
radialfig.write_image('../Visuals/weekly_total_radial.png', engine='kaleido')

In [ ]:
layout = go.Layout(
    autosize=False,
    width=800,
    height=800, 
    
)
radial2fig = go.Figure(layout=layout)
color = {
    'Weekly Total':'rgb(106,81,163)',
    'Weekly Average':'rgb(158,154,200)',
    'Weekly Stdev':'rgb(203,201,226)'
}
for col in ['Weekly Total', 'Weekly Average', 'Weekly Stdev']:
    radial2fig.add_trace(go.Barpolar(
        r=weekly_report[col],
        name=col,
        marker_color=color[col],
    ))

med = weekly_report['Weekly Total'].describe()['50%']



radial2fig.update_traces(text=weekly_report.Week_Num)
radial2fig.update_layout(
    title='Weekly Report',
    #polar_radialaxis_ticksuffix='%',
    polar_angularaxis_rotation=90,
    polar=dict(radialaxis=dict(showticklabels=False, ticks='', linewidth=0))
    
)

radial2fig.show()
radial2fig.write_image('../Visuals/weekly_report_radial.png', engine='kaleido')

In [ ]:
weekly_report['Weekly Total'].describe()#['50%']

In [ ]:
# checking the length of the 1st and 53rd week
# len(ts_df.loc[ts_df.Week_Num == 53].Weekday.unique()) # 4 days (M - Th)
# len(ts_df.loc[ts_df.Week_Num == 1].Weekday.unique()) # 5 days (W - Su)

In [ ]:
T9_Summary = weekly_report.describe()
T9_Summary.drop('Week_Num',axis=1, inplace=True)
weekly_report.to_csv('../Tables/T9:Weekly_Report.csv')
T9_Summary.to_csv('../Tables/T9_Summary.csv')

In [ ]:
month_df = pd.DataFrame(ts_df.groupby(['Month'])['Month'].count())
print(f"Average Incidents per Month of the Year: {ts_df.groupby(['Month'])['Month'].count().mean()}")

fig_month = px.bar(month_df,
    x=month_df.index,
    y='Month',
    labels={'index':'Month','Month':'Count'},
    title='Total Incident Frequency Per Month')
fig_month.show()

fig_month.write_image('../Visuals/Monthly_total.png', engine="kaleido")

In [ ]:
# creating tabular format
month_df.to_csv('../Tables/T4:Total_Frequency_by_Month.csv')
month_df_summary = month_df.describe()
month_df_summary.to_csv('../Tables/T4_Summary.csv')

In [ ]:
# same as previous distribution, but by season
season_df = pd.DataFrame(ts_df.groupby(['Season'])['Season'].count())
print(f"Average Incidents per Season of the Year: {ts_df.groupby(['Season'])['Season'].count().mean()}")

fig_season2 = px.bar(season_df,
    x=season_df.index,
    y='Season',
    labels={'index':'Season','Month':'Count'},
    title='Total Incident Frequency Per Season')
fig_season2.show()

fig_season2.write_image('../Visuals/Season_total.png', engine="kaleido")

In [ ]:
# creating tabular format
season_df_summary = season_df.describe()
season_df.to_csv('../Tables/T5:Total_Frequency_by_Season.csv')
season_df_summary.to_csv('../Tables/T5_Summary.csv')

In [ ]:
# scatter plot with x = hours, y = frequency, color by month
# creating data points in arrays
x = []
y = []
names = []
for i in range(0,24):
    # format hour
    if i < 10:
        ind = f'0{str(i)}:00'
    else:
        ind = f'{str(i)}:00'
    for j in range(1,13):
        date_object = datetime.strptime(str(j),'%m')
        names.append(date_object.strftime('%b'))

        x.append(ind)
        y.append(hours_by_month[j][ind])

df = pd.DataFrame({'Hour':x, 'Frequency': y, 'Month':names})

In [ ]:


hourly_fig = px.scatter(df,
    x='Hour',
    y='Frequency',
    color='Month',  
    symbol='Month',
    color_discrete_sequence=px.colors.qualitative.Dark24 # updated to remove duplicate colors
    # trendline='ols',
    # trendline_scope='overall' 
)

hourly_fig.update_layout(
    yaxis_title='Hourly Instances of DA',
    title_text='Instances of DA per Hour by Month'
)

hourly_fig.show()
hourly_fig.write_image('../Visuals/DA_Monthly_by_Hour.png', engine='kaleido')

In [ ]:
# creating tabular format
df_summary = df.describe()
df.to_csv('../Tables/T6:Total_Frequency_Per_Month_by_Hour.csv')
df_summary.to_csv('../Tables/T6_Summary.csv')

In [ ]:
# Same as previous but for Seasons
# scatter plot with x = hours, y = frequency, color by season
# creating data points in arrays
x_s = []
y_s = []
names_s = []
for i in range(0,24):
    # format hour
    if i < 10:
        ind = f'0{str(i)}:00'
    else:
        ind = f'{str(i)}:00'
    for j in ['Winter', 'Spring','Summer','Fall']:
        names_s.append(j)
        x_s.append(ind)
        y_s.append(hours_by_season[j][ind])

df_s = pd.DataFrame({'Hour':x_s, 'Frequency': y_s, 'Season':names_s})

hourly_season_fig = px.scatter(df_s,
    x='Hour',
    y='Frequency',
    color='Season',  
    symbol='Season',
    # trendline='ols',
    # trendline_scope='overall' 
)

hourly_season_fig.update_layout(
    yaxis_title='Hourly Instances of DA',
    title_text='Instances of DA per Hour by Month'
)

hourly_season_fig.show()
hourly_season_fig.write_image('../Visuals/DA_Season_by_Hour.png', engine='kaleido')

In [ ]:
# creating tabular format
df_s_summary = df_s.describe()
df_s.to_csv('../Tables/T7:Total_Frequency_Per_Season_by_Hour.csv')
df_s_summary.to_csv('../Tables/T7_Summary.csv')

In [ ]:
df2 = df
df2.Hour = df2.Hour.apply(change_time_format2)

In [ ]:


overall_hourly_fig = px.scatter(df2,
    x='Hour',
    y='Frequency',
    # color='Month',  
    # symbol='Month',
    trendline='ols',
    # trendline_options=dict(window=5)
)

overall_hourly_fig.update_layout(
    yaxis_title='Hourly Instances of DA',
    title_text='Overall DA per Hour (5-pt)'
)

overall_hourly_fig.show()
overall_hourly_fig.write_image('../Visuals/DA_Overall_by_Hour.png', engine='kaleido')